In [ ]:
#Functions for cluster work.
import os,ast

os.chdir(r"/home/")
import pandas as pd
import numpy as np

def splitBlogs(inputDataFrame):
    id_list = list(set(inputDataFrame['hardcode_id'].tolist()))
    np.random.seed(1)
    np.random.shuffle(id_list)
    train_id_list = id_list[:int(len(id_list) * 0.8)]
    test_id_list = id_list[int(len(id_list) * 0.8):]
    out_train = inputDataFrame[inputDataFrame['hardcode_id'].isin(train_id_list)]
    out_test = inputDataFrame[inputDataFrame['hardcode_id'].isin(test_id_list)]
    return out_train, out_test

#Generate the feature for each article and unsupervised clustering.
def feature_build_and_self_cluster(input_DF):
    input_DF = input_DF.copy()
    feature_total = []
    t = 0
    for i in input_DF.index:
        thisLine = [input_DF.loc[i, 'IOC Statistics: IOC Words Number in CTI sentences'],
                    input_DF.loc[i, 'IOC Statistics: IOC Words Number in All sentences'],
                    input_DF.loc[i, 'Sentence Statistics:  Rate of CTI Sentences '],
                    input_DF.loc[i, 'Sentence Statistics:  CTi Sentences'],
                    input_DF.loc[i, 'Sentence Statistics:  All Sentences'],
                    input_DF.loc[i, 'Sentence Statistics:  tactic Sentences']]
        thisLine.extend((input_DF.loc[i, 'Tactics Statistics: CTI Sentences']))
        thisLine.extend((input_DF.loc[i, 'Tactics Statistics: CTI Sentences (Exist or Not)']))
        if t == 0:
            weight_dimension = len(thisLine)
            print(weight_dimension)
            t = 1
        thisLine.extend((input_DF.loc[i, 'Model Result Embedding_PCA']))
        thisLine.extend((input_DF.loc[i, ' CTI Sentence Embedding_PCA']))
        feature_total.append(thisLine)
    for weight_value in [30]:
        for cluster_number in [100]:
            for min_samples_value in [3]:
                import matplotlib.pyplot as plt
                import hdbscan
                from sklearn.preprocessing import StandardScaler
                X = StandardScaler().fit_transform(feature_total)
                X = X.tolist()
                #convert X to list of list
                for i in X:
                    for j in range(len(i)):
                        if j <= weight_dimension:
                            i[j] = i[j] * weight_value
                feature_total = np.array(X)
                clusterer = hdbscan.HDBSCAN(min_cluster_size=cluster_number, min_samples=min_samples_value,
                                            cluster_selection_method='leaf', prediction_data=True).fit(feature_total)
                y_pred = hdbscan.HDBSCAN(min_cluster_size=cluster_number, min_samples=min_samples_value,
                                         cluster_selection_method='leaf').fit_predict(feature_total)
                from sklearn.manifold import TSNE
                projection = TSNE(n_components=2).fit_transform(feature_total)
                from matplotlib.axes._axes import _log as matplotlib_axes_logger
                matplotlib_axes_logger.setLevel('ERROR')
                plt.figure(figsize=(40, 40))
                from matplotlib.cm import get_cmap
                colors = get_cmap("tab20").colors + get_cmap("tab20b").colors
                for i, c in zip(list(set(y_pred)), colors):
                    plt.scatter(projection[y_pred == i, 0], projection[y_pred == i, 1], c=c)
                plt.legend(list(set(y_pred)))
                from collections import Counter
                print(Counter(y_pred))
                input_DF['HDBSCAN cluster result'] = y_pred
    #deepcopy input_DF
    return input_DF, clusterer


def findvalue(in_dict, x):
    if x in in_dict.keys():
        return str(in_dict[x])
    else:
        return 'NaN'


def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)


def probility(x):
    return softmax(x)[1]


def build_feature(input_DF):
    input_DF = input_DF.copy()
    feature_total = []
    t = 0
    for i in input_DF.index:
        thisLine = [input_DF.loc[i, 'IOC Statistics: IOC Words Number in CTI sentences'],
                    input_DF.loc[i, 'IOC Statistics: IOC Words Number in All sentences'],
                    input_DF.loc[i, 'Sentence Statistics:  Rate of CTI Sentences '],
                    input_DF.loc[i, 'Sentence Statistics:  CTi Sentences'],
                    input_DF.loc[i, 'Sentence Statistics:  All Sentences'],
                    input_DF.loc[i, 'Sentence Statistics:  tactic Sentences']]
        thisLine.extend((input_DF.loc[i, 'Tactics Statistics: CTI Sentences']))
        thisLine.extend((input_DF.loc[i, 'Tactics Statistics: CTI Sentences (Exist or Not)']))
        if t == 0:
            weight_dimension = len(thisLine)
            print(weight_dimension)
            t = 1
        thisLine.extend((input_DF.loc[i, 'Model Result Embedding_PCA']))
        thisLine.extend((input_DF.loc[i, ' CTI Sentence Embedding_PCA']))
        feature_total.append(thisLine)
    for weight_value in [30]:
        print('weight_value', weight_value)
        from sklearn.preprocessing import StandardScaler
        X = StandardScaler().fit_transform(feature_total)
        X = X.tolist()
        #convert X to list of list
        for i in X:
            for j in range(len(i)):
                if j <= weight_dimension:
                    i[j] = i[j] * weight_value
        #convert X to array
        feature_total = np.array(X)
        return feature_total

In [ ]:
#Import the data
blog_data = pd.read_csv('Blog Clustering Results Training Set.csv', encoding='utf-8-sig',
                        converters={'Sentence Statistics:  Rate of CTI Sentences ': ast.literal_eval,
                                    'Tactics Statistics: CTI Sentences': ast.literal_eval,
                                    'Tactics Statistics: CTI Sentences (Exist or Not)': ast.literal_eval,
                                    'Model Result Embedding_PCA': ast.literal_eval,
                                    ' CTI Sentence Embedding_PCA': ast.literal_eval,
                                    'Sentence Statistics:  All Sentences': ast.literal_eval,
                                    'Sentence Statistics:  tactic Sentences': ast.literal_eval})
df_train, df_test = splitBlogs(blog_data)

In [ ]:
#Generate the feature vector and cluster the blogs in the training set
df_train, HDBSCANCluster =feature_build_and_self_cluster(df_train)

In [ ]:
#Calculation of silhouette coefficient for the training set
import sklearn
feature_total = df_train['Feature Vector'].values.tolist()  # type: ignore
sample_silhouette_values = sklearn.metrics.silhouette_samples(feature_total,
                                                              df_train['HDBSCAN Cluster'].values.tolist())
df_train['silhouette'] = sample_silhouette_values
for i in range(-1, 19):
    thisDF = df_train[df_train['HDBSCAN Cluster'] == i]
    print(i, np.mean(thisDF['silhouette']))
print(sklearn.metrics.silhouette_score(feature_total, df_train['HDBSCAN Cluster'].values.tolist(), metric='euclidean'))

In [ ]:
#Generate the 2D visualization of the training set.
from collections import Counter
from matplotlib import pyplot as plt
from sklearn.manifold import TSNE
from matplotlib.cm import get_cmap

colors = get_cmap("tab20").colors + get_cmap("tab20b").colors  # type: ignore
plt.figure(figsize=(40, 40))
finlegend = []
feature_vectors = df_train['Feature Vector'].values.tolist()  # type: ignore
y_pred = df_train['HDBSCAN Cluster'].values.tolist()  # type: ignore
cm = plt.get_cmap('gist_rainbow')  # type: ignore
projection = TSNE(n_components=2).fit_transform(feature_vectors)
for i, c in zip(list(set(y_pred)), colors):
    if i != -1:
        plt.scatter(projection[y_pred == i, 0], projection[y_pred == i, 1], c=c)
        finlegend.append(i)
plt.legend(finlegend)
print(Counter(y_pred))
plt.savefig('output.png')